In [ ]:
# importing necessary libraries for data loading and EDA
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, auc, precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix


warnings.filterwarnings('ignore')

In [29]:
# Loading the train data
df_train = pd.read_csv(r'/home/ducanh/Credit Card Transactions Fraud Detection/Datasets/fraudTrain.csv')
df_test = pd.read_csv(r'/home/ducanh/Credit Card Transactions Fraud Detection/Datasets/fraudTest.csv')
df = pd.concat([df_train, df_test])

In [30]:
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df['trans_hour'] = df['trans_date_trans_time'].dt.time.apply(lambda x: str(x)[:2])

df['dob'] = pd.to_datetime(df['dob'])
df['cust_age'] = df['dob'].dt.year.apply(lambda x: 2021-x)
df['cust_age_groups'] = df['cust_age'].apply(lambda x: 'below 10' if x<10 else ('10-20' if x>=10 and x<20 else ('20-30' if x>=20 and x<30 else('30-40' if x>=30 and x<40 else('40-50' if x>=40 and x<50 else('50-60' if x>=50 and x<60 else('60-70' if x>=60 and x<70 else ('70-80' if x>=70 and x<80 else 'Above 80'))))))))

In [31]:
drop_col = ['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant','first', 'last', 'street', 'city', 'state', 'lat',
       'long','dob', 'unix_time', 'cust_age', 'merch_lat',
       'merch_long', 'city_pop']
df.drop(drop_col, axis = 1, inplace = True)

In [32]:
age_piv_2 = pd.pivot_table(data = df,
                           index = 'cust_age_groups',
                           columns = 'is_fraud',
                           values = 'amt',
                           aggfunc = np.mean)
age_piv_2.sort_values(by = 1, ascending = True, inplace = True)
age_dic = {k:v for (k,v) in zip(age_piv_2.index.values, age_piv_2.reset_index().index.values)}
df['cust_age_groups'] = df['cust_age_groups'].map(age_dic)

In [33]:
merch_cat = df[df['is_fraud'] == 1].groupby('category')['amt'].mean().sort_values(ascending = True)
merch_cat_dic = {k:v for (k,v) in zip(merch_cat.index.values,merch_cat.reset_index().index.values)}
df['category'] = df['category'].map(merch_cat_dic)

In [34]:
job_txn_piv_2 = pd.pivot_table(data = df,
                               index = 'job',
                               columns = 'is_fraud',
                               values= 'amt',
                               aggfunc = np.mean)
job_cat_dic = {k:v for (k,v) in zip(job_txn_piv_2.index.values, job_txn_piv_2.reset_index().index.values)}
df['job'] = df['job'].map(job_cat_dic)

In [35]:
df['trans_hour'] = df['trans_hour'].astype('int')
df = pd.get_dummies(data  = df, columns = ['gender'], drop_first = True, dtype = 'int')

# Train_test_split

In [36]:
train,test = train_test_split(df, test_size=0.33, random_state=42, stratify = df['is_fraud'])
# visualizing class imbalance
df['is_fraud'].value_counts()
# check
print(train.shape)
print(test.shape)

# let's drop transaction number columns from both the training and testing data
train.drop('trans_num',axis = 1, inplace = True)
test.drop('trans_num',axis = 1, inplace = True)
# splitting data into dependent and independent features respectively
y_train = train['is_fraud']
X_train = train.drop('is_fraud',axis = 1)

y_test = test['is_fraud']
X_test = test.drop('is_fraud',axis = 1)

print('Shape of training data: ',(X_train.shape,y_train.shape))
print('Shape of testing data: ',(X_test.shape,y_test.shape))

(1241103, 9)
(611291, 9)
Shape of training data:  ((1241103, 7), (1241103,))
Shape of testing data:  ((611291, 7), (611291,))


In [37]:
sc = StandardScaler()
# scaling the training and testing data
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)
# convert them into dataframes
X_train_sc = pd.DataFrame(data = X_train_sc, columns = X_train.columns)
X_test_sc = pd.DataFrame(data = X_test_sc, columns = X_test.columns)

# KNN

In [38]:
N = X_train.shape[0]  # Số lượng mẫu trong tập train
k_initial = int(np.sqrt(N))  
if k_initial % 2 == 0:
    k_initial += 1  # Đảm bảo k là số lẻ

In [ ]:
knn = KNeighborsClassifier(n_neighbors=k_initial)

# 🔥 Huấn luyện mô hình trên tập train
knn.fit(X_train_sc, y_train)

# 📌 Dự đoán xác suất (Xác suất thuộc lớp 1 - fraud)
y_pred_train_proba = knn.predict_proba(X_train_sc)
y_pred_test_proba = knn.predict_proba(X_test_sc)

# Lưu kết quả vào DataFrame
y_train_results = pd.DataFrame(y_pred_train_proba, columns=['pred_not_fraud', 'pred_fraud'])
y_test_results = pd.DataFrame(y_pred_test_proba, columns=['pred_not_fraud', 'pred_fraud'])

y_train_results['y_train_actual'] = y_train.values
y_test_results['y_test_actual'] = y_test.values

In [ ]:
numbers = [0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
for i in numbers:
    y_train_results[i] = y_train_results['pred_fraud'].map(lambda x: 1 if x > i else 0)
    y_test_results[i] = y_test_results['pred_fraud'].map(lambda x: 1 if x > i else 0)

In [ ]:
# 📌 Tính Accuracy, Precision, Recall, F1-score trên tập Train
cutoff_df = pd.DataFrame(columns=['Threshold', 'Accuracy', 'Precision', 'Recall', 'F1_score'])

for i in numbers:
    cm1 = confusion_matrix(y_train_results['y_train_actual'], y_train_results[i])
    TP, FP, FN, TN = cm1[1, 1], cm1[0, 1], cm1[1, 0], cm1[0, 0]

    # Tính Accuracy, Precision, Recall, F1-score
    accuracy = (TP + TN) / (TP + FP + FN + TN)
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score_value = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    cutoff_df.loc[i] = [i, accuracy, precision, recall, f1_score_value]

print("📌 Train Set Performance:")
print(cutoff_df)

# 📌 Tìm Threshold có Accuracy cao nhất trên tập Train
best_idx = cutoff_df['Accuracy'].idxmax()
best_threshold = cutoff_df.loc[best_idx, 'Threshold']
best_accuracy = cutoff_df.loc[best_idx, 'Accuracy']
best_f1_score = cutoff_df.loc[best_idx, 'F1_score']
best_precision = cutoff_df.loc[best_idx, 'Precision']
best_recall = cutoff_df.loc[best_idx, 'Recall']
best_auc = roc_auc_score(y_train_results['y_train_actual'], y_train_results['pred_fraud'])

# ✅ In kết quả tối ưu trên Train
print(f'Best Threshold: {best_threshold:.4f}')
print(f'Best Accuracy: {best_accuracy:.4f}')
print(f'Best Precision: {best_precision:.4f}')
print(f'Best Recall: {best_recall:.4f}')
print(f'Best F1 Score: {best_f1_score:.4f}')
print(f'Best ROC_AUC Score: {best_auc:.4f}')

📌 Train Set Performance:
      Threshold  Accuracy  Precision   Recall  F1_score
0.10       0.10  0.998754   0.807039  1.00000  0.893217
0.15       0.15  0.998754   0.807039  1.00000  0.893217
0.20       0.20  0.998754   0.807039  1.00000  0.893217
0.30       0.30  0.998754   0.807039  1.00000  0.893217
0.40       0.40  0.998754   0.807039  1.00000  0.893217
0.50       0.50  0.998306   1.000000  0.67476  0.805799
0.60       0.60  0.998306   1.000000  0.67476  0.805799
0.70       0.70  0.998306   1.000000  0.67476  0.805799
0.80       0.80  0.998306   1.000000  0.67476  0.805799
0.90       0.90  0.998306   1.000000  0.67476  0.805799
Best Threshold: 0.1000
Best Accuracy: 0.9988
Best Precision: 0.8070
Best Recall: 1.0000
Best F1 Score: 0.8932
Best ROC_AUC Score: 0.9998


In [ ]:
# 📌 Tính Accuracy, Precision, Recall, F1-score trên tập Test
cutoff_df = pd.DataFrame(columns=['Threshold', 'Accuracy', 'Precision', 'Recall', 'F1_score'])

for i in numbers:
    cm1 = confusion_matrix(y_test_results['y_test_actual'], y_test_results[i])
    TP, FP, FN, TN = cm1[1, 1], cm1[0, 1], cm1[1, 0], cm1[0, 0]

    # Tính Accuracy, Precision, Recall, F1-score
    accuracy = (TP + TN) / (TP + FP + FN + TN)
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score_value = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    cutoff_df.loc[i] = [i, accuracy, precision, recall, f1_score_value]

print("📌 Test Set Performance:")
print(cutoff_df)

# 📌 Tìm Threshold có Accuracy cao nhất trên tập Test
best_idx = cutoff_df['Accuracy'].idxmax()
best_threshold = cutoff_df.loc[best_idx, 'Threshold']
best_accuracy = cutoff_df.loc[best_idx, 'Accuracy']
best_f1_score = cutoff_df.loc[best_idx, 'F1_score']
best_precision = cutoff_df.loc[best_idx, 'Precision']
best_recall = cutoff_df.loc[best_idx, 'Recall']
best_auc = roc_auc_score(y_test_results['y_test_actual'], y_test_results['pred_fraud'])

# ✅ In kết quả tối ưu trên Test
print(f'Best Threshold: {best_threshold:.4f}')
print(f'Best Accuracy: {best_accuracy:.4f}')
print(f'Best Precision: {best_precision:.4f}')
print(f'Best Recall: {best_recall:.4f}')
print(f'Best F1 Score: {best_f1_score:.4f}')
print(f'Best ROC_AUC Score: {best_auc:.4f}')

📌 Test Set Performance:
      Threshold  Accuracy  Precision    Recall  F1_score
0.10       0.10  0.996316   0.622088  0.746311  0.678561
0.15       0.15  0.996316   0.622088  0.746311  0.678561
0.20       0.20  0.996316   0.622088  0.746311  0.678561
0.30       0.30  0.996316   0.622088  0.746311  0.678561
0.40       0.40  0.996316   0.622088  0.746311  0.678561
0.50       0.50  0.997422   0.900050  0.568289  0.696690
0.60       0.60  0.997422   0.900050  0.568289  0.696690
0.70       0.70  0.997422   0.900050  0.568289  0.696690
0.80       0.80  0.997422   0.900050  0.568289  0.696690
0.90       0.90  0.997422   0.900050  0.568289  0.696690
Best Threshold: 0.5000
Best Accuracy: 0.9974
Best Precision: 0.9000
Best Recall: 0.5683
Best F1 Score: 0.6967
Best ROC_AUC Score: 0.8725
